<a href="https://colab.research.google.com/github/deepampatel/randomnlp/blob/main/MarathiRoberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Oct  9 06:57:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download Data

In [ ]:
!wget https://oscar-public.huma-num.fr/shuffled/mr_dedup.txt.gz
!gzip -d mr_dedup.txt.gz

--2020-10-09 06:57:40--  https://oscar-public.huma-num.fr/shuffled/mr_dedup.txt.gz
Resolving oscar-public.huma-num.fr (oscar-public.huma-num.fr)... 134.158.33.192
Connecting to oscar-public.huma-num.fr (oscar-public.huma-num.fr)|134.158.33.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 313843639 (299M) [application/x-gzip]
Saving to: ‘mr_dedup.txt.gz’

mr_dedup.txt.gz     100%[===================>] 299.30M  27.6MB/s    in 12s     

2020-10-09 06:57:52 (25.5 MB/s) - ‘mr_dedup.txt.gz’ saved [313843639/313843639]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import gc
import glob
import torch
import pickle
import joblib
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

In [ ]:
!pip install tokenizers
!git clone https://github.com/huggingface/transformers
!pip install transformers/.


     |████████████████████████████████| 3.0MB 1.7MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 45802 (delta 4), reused 3 (delta 0), pack-reused 45780
Receiving objects: 100% (45802/45802), 32.76 MiB | 28.77 MiB/s, done.
Resolving deltas: 100% (31751/31751), done.
Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 1.7MB/s 
     |████████████████████████████████| 2.9MB 9.5MB/s 
     |████████████████████████████████| 890kB 19.1MB/s 
  Created wheel for transformers: filename=transformers-3.3.1-cp36-none-any.whl size=1133177 sha256=40975d70da28d40e091875da3513a1568930807888fb91fb1c2293413cd4b34a
  Stored in directory: /tmp/pip-ephem-wheel-cache-6nxmc2d_/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c69

## Create the Tokenizer 

In [ ]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = ["mr_dedup.txt.gz"]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(
)

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


In [ ]:
tokenizer.save_model("/content/drive/My Drive/marathiroberta") #Saves it to drive

['martbertochar/vocab.json', 'martbertochar/merges.txt']

## Load the saved Tokenizer and test

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers.decoders import ByteLevel

tokenizer = ByteLevelBPETokenizer(
    "/content/drive/My Drive/marathiroberta/vocab.json",
    "/content/drive/My Drive/marathiroberta/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)
op = tokenizer.encode("एखाद्या कुशल चित्रकाराने कुंचल्याच्या अवघ्या चार-सहा फटकाऱ्यांसरशी एखादे सुरेख चित्र निर्माण करावे तद्वत अवघ्या आठ ओळीत बालकवींनी एक सुंदर निसर्गचित्र शब्दांच्या कुंचल्याने या कवितेत रेखाटलेले आहे.  ")
decoder = ByteLevel()
tokenizer.decode(op)

## Create Roberta model and load the  tokenizer 

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaTokenizerFast


tokenizer = RobertaTokenizerFast.from_pretrained("/content/drive/My Drive/marathiroberta", max_len=512)

In [ ]:
tokenizer.save_pretrained("/content/drive/My Drive/marathiroberta")

('/content/drive/My Drive/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/tokenizer/special_tokens_map.json',
 ('/content/drive/My Drive/tokenizer/vocab.json',
  '/content/drive/My Drive/tokenizer/merges.txt'),
 '/content/drive/My Drive/tokenizer/added_tokens.json')

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
# model = RobertaForMaskedLM.from_pretrained('/content/drive/My Drive/marathiroberta/checkpoint-90000', return_dict=True)  #To load from a checkpoint

In [ ]:
model.num_parameters()

126031648

## Create a lazy data loader

In [ ]:
from torch.utils.data.dataset import Dataset
import os
from transformers.tokenization_utils import PreTrainedTokenizer
import torch
import logging
logger = logging.getLogger(__name__)

class LazyLineByLineTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, file_path: str, block_size: int):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)
        logger.info("Creating features from dataset file at %s", file_path)

        with open(file_path, encoding="utf-8") as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
        self.block_size = block_size
        # batch_encoding = tokenizer(lines, add_special_tokens=True, truncation=True, max_length=block_size)
        self.examples = lines
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        
        return torch.tensor(self.tokenizer(self.examples[i], add_special_tokens=True, truncation=True, max_length=self.block_size)["input_ids"], dtype=torch.long)

## Create a dataloader and use that to create a trainer

In [ ]:
%%time
from transformers import LineByLineTextDataset

#For lazy loading the data
dataset = LazyLineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="mr_dedup.txt",
    block_size=128,
)

#For loading the whole dataset in memory
# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path="mr_dedup.txt",
#     block_size=128,
# )

CPU times: user 8.97 s, sys: 4.46 s, total: 13.4 s
Wall time: 13.4 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.20
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/marathiroberta",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=48,
    save_steps=5000,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)


/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:246: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


{'loss': 0.4169935302734375, 'learning_rate': 4.9734073672229845e-05, 'epoch': 0.015955579666209273}
{'loss': 0.4232584228515625, 'learning_rate': 4.9468147344459694e-05, 'epoch': 0.031911159332418546}
{'loss': 0.4242197265625, 'learning_rate': 4.9202221016689536e-05, 'epoch': 0.04786673899862782}
{'loss': 0.4336846923828125, 'learning_rate': 4.8936294688919385e-05, 'epoch': 0.06382231866483709}
{'loss': 0.4378968505859375, 'learning_rate': 4.8670368361149234e-05, 'epoch': 0.07977789833104637}
{'loss': 0.4377626953125, 'learning_rate': 4.840444203337908e-05, 'epoch': 0.09573347799725564}
{'loss': 0.443458740234375, 'learning_rate': 4.813851570560892e-05, 'epoch': 0.11168905766346492}
{'loss': 0.442999267578125, 'learning_rate': 4.787258937783877e-05, 'epoch': 0.12764463732967418}
{'loss': 0.475087890625, 'learning_rate': 4.760666305006861e-05, 'epoch': 0.14360021699588346}
{'loss': 0.503595703125, 'learning_rate': 4.734073672229846e-05, 'epoch': 0.15955579666209274}
{'loss': 0.49837597

In [ ]:
trainer.save_model("/content/drive/My Drive/marathiroberta")

## Load saved model and test

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/drive/My Drive/marathiroberta",
    tokenizer="/content/drive/My Drive/marathiroberta"
)

In [ ]:
fill_mask("राष्ट्रवादीचे सर्वेसर्वा <mask> पवार यांनी मुंडेंना राष्ट्रवादीचा प्रस्ताव नसल्याचे म्हटले आहे.")

[{'score': 0.9996238946914673,
  'sequence': '<s>राष्ट्रवादीचे सर्वेसर्वा शरद पवार यांनी मुंडेंना राष्ट्रवादीचा प्रस्ताव नसल्याचे म्हटले आहे.</s>',
  'token': 1405,
  'token_str': 'Ġà¤¶à¤°à¤¦'},
 {'score': 0.00015381297271233052,
  'sequence': '<s>राष्ट्रवादीचे सर्वेसर्वाशरद पवार यांनी मुंडेंना राष्ट्रवादीचा प्रस्ताव नसल्याचे म्हटले आहे.</s>',
  'token': 4942,
  'token_str': 'à¤¶à¤°à¤¦'},
 {'score': 8.309441909659654e-05,
  'sequence': '<s>राष्ट्रवादीचे सर्वेसर्वा भरत पवार यांनी मुंडेंना राष्ट्रवादीचा प्रस्ताव नसल्याचे म्हटले आहे.</s>',
  'token': 1207,
  'token_str': 'Ġà¤Ńà¤°à¤¤'},
 {'score': 4.730443106382154e-05,
  'sequence': '<s>राष्ट्रवादीचे सर्वेसर्वा छगन पवार यांनी मुंडेंना राष्ट्रवादीचा प्रस्ताव नसल्याचे म्हटले आहे.</s>',
  'token': 3803,
  'token_str': 'Ġà¤Ľà¤Ĺà¤¨'},
 {'score': 2.4680019123479724e-05,
  'sequence': '<s>राष्ट्रवादीचे सर्वेसर्वा करण पवार यांनी मुंडेंना राष्ट्रवादीचा प्रस्ताव नसल्याचे म्हटले आहे.</s>',
  'token': 345,
  'token_str': 'Ġà¤ķà¤°à¤£'}]